In [4]:
import requests

API_KEY = '8cd2572cd7ed88b8fa239ea6d041f9ea'
USER_AGENT = 'thisisbigcat'

headers = {
    'user_agent': USER_AGENT
}

payload = {
    'api_key': API_KEY,
    'method': 'chart.gettopartists',
    'format': 'json'
}

r = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=payload)
r.status_code

200

In [5]:
def lastfm_get(payload):
    # define headers and URL
    headers = {'user-agent': USER_AGENT}
    url = 'https://ws.audioscrobbler.com/2.0/'

    # Add API key and format to the payload
    payload['api_key'] = API_KEY
    payload['format'] = 'json'

    response = requests.get(url, headers=headers, params=payload)
    return response

In [6]:
r = lastfm_get({
    'method': 'chart.gettopartists'
})

r.status_code

200

In [9]:
import json

def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

jprint(r.json())

{
    "artists": {
        "@attr": {
            "page": "1",
            "perPage": "50",
            "total": "5289405",
            "totalPages": "105789"
        },
        "artist": [
            {
                "image": [
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png",
                        "size": "small"
                    },
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png",
                        "size": "medium"
                    },
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png",
                        "size": "large"
                    },
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png",
                        "s

In [5]:
jprint(r.json()['artists']['@attr'])

{
    "page": "1",
    "perPage": "50",
    "total": "5289405",
    "totalPages": "105789"
}


In [7]:
import requests_cache

requests_cache.install_cache()

In [10]:
import time
from IPython.core.display import clear_output

responses = []

page = 1
total_pages = 99999 # this is just a dummy number so the loop starts

while page <= total_pages:
    payload = {
        'method': 'chart.gettopartists',
        'limit': 500,
        'page': page
    }

    # print some output so we can see the status
    print("Requesting page {}/{}".format(page, total_pages))
    # clear the output to make things neater
    clear_output(wait = True)

    # make the API call
    response = lastfm_get(payload)

    # if we get an error, print the response and halt the loop
    if response.status_code != 200:
        print(response.text)
        break

    # extract pagination info
    page = int(response.json()['artists']['@attr']['page'])
    total_pages = int(response.json()['artists']['@attr']['totalPages'])

    # append response
    responses.append(response)

    # if it's not a cached result, sleep
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)

    # increment the page number
    page += 1

Requesting page 10617/10617


In [11]:
import pandas as pd

r0 = responses[0]
r0_json = r0.json()
r0_artists = r0_json['artists']['artist']
r0_df = pd.DataFrame(r0_artists)
r0_df.head()

,name,playcount,listeners,mbid,url,streamable,image
0,The Weeknd,481735252,3330059,c8b03190-306c-4120-bb0b-6f2ebfc06ea9,https://www.last.fm/music/The+Weeknd,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
1,Taylor Swift,1411778934,4025748,20244d07-534f-4eff-b4d4-930878889970,https://www.last.fm/music/Taylor+Swift,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
2,Lana Del Rey,667021372,3456177,b7539c32-53e7-4908-bda3-81449c367da6,https://www.last.fm/music/Lana+Del+Rey,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
3,Kanye West,780600731,6046182,164f0d73-1234-4e2c-8743-d77bf2191051,https://www.last.fm/music/Kanye+West,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
4,Kendrick Lamar,391661771,3059541,381086ea-f511-4aba-bdf9-71c753dc5077,https://www.last.fm/music/Kendrick+Lamar,0,[{'#text': 'https://lastfm.freetls.fastly.net/...


In [12]:
frames = [pd.DataFrame(r.json()['artists']['artist']) for r in responses]
artists = pd.concat(frames)
artists.head()

,name,playcount,listeners,mbid,url,streamable,image
0,The Weeknd,481735252,3330059,c8b03190-306c-4120-bb0b-6f2ebfc06ea9,https://www.last.fm/music/The+Weeknd,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
1,Taylor Swift,1411778934,4025748,20244d07-534f-4eff-b4d4-930878889970,https://www.last.fm/music/Taylor+Swift,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
2,Lana Del Rey,667021372,3456177,b7539c32-53e7-4908-bda3-81449c367da6,https://www.last.fm/music/Lana+Del+Rey,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
3,Kanye West,780600731,6046182,164f0d73-1234-4e2c-8743-d77bf2191051,https://www.last.fm/music/Kanye+West,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
4,Kendrick Lamar,391661771,3059541,381086ea-f511-4aba-bdf9-71c753dc5077,https://www.last.fm/music/Kendrick+Lamar,0,[{'#text': 'https://lastfm.freetls.fastly.net/...


In [13]:
artists = artists.drop('image', axis=1)
artists.head()

,name,playcount,listeners,mbid,url,streamable
0,The Weeknd,481735252,3330059,c8b03190-306c-4120-bb0b-6f2ebfc06ea9,https://www.last.fm/music/The+Weeknd,0
1,Taylor Swift,1411778934,4025748,20244d07-534f-4eff-b4d4-930878889970,https://www.last.fm/music/Taylor+Swift,0
2,Lana Del Rey,667021372,3456177,b7539c32-53e7-4908-bda3-81449c367da6,https://www.last.fm/music/Lana+Del+Rey,0
3,Kanye West,780600731,6046182,164f0d73-1234-4e2c-8743-d77bf2191051,https://www.last.fm/music/Kanye+West,0
4,Kendrick Lamar,391661771,3059541,381086ea-f511-4aba-bdf9-71c753dc5077,https://www.last.fm/music/Kendrick+Lamar,0


In [14]:
artists.info()
artists.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13200 entries, 0 to 999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        13200 non-null  object
 1   playcount   13200 non-null  object
 2   listeners   13200 non-null  object
 3   mbid        13200 non-null  object
 4   url         13200 non-null  object
 5   streamable  13200 non-null  object
dtypes: object(6)
memory usage: 721.9+ KB


,name,playcount,listeners,mbid,url,streamable
count,13200,13200,13200,13200,13200,13200
unique,10000,9997,9922,5497,10000,1
top,The Weeknd,1204041,111091,,https://www.last.fm/music/The+Weeknd,0
freq,2,3,4,5960,2,13200


In [15]:
artist_counts = [len(r.json()['artists']['artist']) for r in responses]
pd.Series(artist_counts).value_counts()

0       10597
1000       10
500         6
50          4
dtype: int64

In [16]:
print(artist_counts[:50])

[500, 1000, 50, 1000, 50, 1000, 50, 1000, 50, 1000, 500, 1000, 500, 1000, 500, 1000, 500, 1000, 500, 1000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [17]:
artists = artists.drop_duplicates().reset_index(drop=True)
artists.describe()

,name,playcount,listeners,mbid,url,streamable
count,10000,10000,10000,10000,10000,10000
unique,10000,9997,9922,5497,10000,1
top,The Weeknd,451263,268305,,https://www.last.fm/music/The+Weeknd,0
freq,1,2,2,4485,1,10000


In [18]:
r = lastfm_get({
    'method': 'artist.getTopTags',
    'artist':  'Lana Del Rey'
})

jprint(r.json())

{
    "toptags": {
        "@attr": {
            "artist": "Lana Del Rey"
        },
        "tag": [
            {
                "count": 100,
                "name": "female vocalists",
                "url": "https://www.last.fm/tag/female+vocalists"
            },
            {
                "count": 94,
                "name": "indie",
                "url": "https://www.last.fm/tag/indie"
            },
            {
                "count": 89,
                "name": "indie pop",
                "url": "https://www.last.fm/tag/indie+pop"
            },
            {
                "count": 81,
                "name": "pop",
                "url": "https://www.last.fm/tag/pop"
            },
            {
                "count": 67,
                "name": "alternative",
                "url": "https://www.last.fm/tag/alternative"
            },
            {
                "count": 15,
                "name": "dream pop",
                "url": "https://www.last.fm/tag/

In [21]:
tags = [t['name'] for t in r.json()['toptags']['tag'][:3]]
', '.join(tags)

'female vocalists, indie, indie pop'

In [22]:
def lookup_tags(artist):
    response = lastfm_get({
        'method': 'artist.getTopTags',
        'artist':  artist
    })

    # if there's an error, just return nothing
    if response.status_code != 200:
        return None

    # extract the top three tags and turn them into a string
    tags = [t['name'] for t in response.json()['toptags']['tag'][:3]]
    tags_str = ', '.join(tags)

    # rate limiting
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)
    return tags_str

In [23]:
lookup_tags("Billie Eilish")

'pop, indie pop, female vocalists'

In [26]:
from tqdm import tqdm
tqdm.pandas()

artists['tags'] = artists['name'].progress_apply(lookup_tags)

100%|██████████| 10000/10000 [49:31<00:00,  3.37it/s] 


In [27]:
artists.head()

,name,playcount,listeners,mbid,url,streamable,tags
0,The Weeknd,481735252,3330059,c8b03190-306c-4120-bb0b-6f2ebfc06ea9,https://www.last.fm/music/The+Weeknd,0,"rnb, electronic, dubstep"
1,Taylor Swift,1411778934,4025748,20244d07-534f-4eff-b4d4-930878889970,https://www.last.fm/music/Taylor+Swift,0,"country, pop, female vocalists"
2,Lana Del Rey,667021372,3456177,b7539c32-53e7-4908-bda3-81449c367da6,https://www.last.fm/music/Lana+Del+Rey,0,"female vocalists, indie, indie pop"
3,Kanye West,780600731,6046182,164f0d73-1234-4e2c-8743-d77bf2191051,https://www.last.fm/music/Kanye+West,0,"Hip-Hop, rap, hip hop"
4,Kendrick Lamar,391661771,3059541,381086ea-f511-4aba-bdf9-71c753dc5077,https://www.last.fm/music/Kendrick+Lamar,0,"Hip-Hop, rap, seen live"


In [28]:
artists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        10000 non-null  object
 1   playcount   10000 non-null  object
 2   listeners   10000 non-null  object
 3   mbid        10000 non-null  object
 4   url         10000 non-null  object
 5   streamable  10000 non-null  object
 6   tags        10000 non-null  object
dtypes: object(7)
memory usage: 547.0+ KB


In [32]:
artists = artists.sort_values("listeners", ascending=False)
artists.head(10)

,name,playcount,listeners,mbid,url,streamable,tags
9448,Fana Hues,784783,99994,,https://www.last.fm/music/Fana+Hues,0,"soul, rnb, pop"
6268,punctual,663140,99992,,https://www.last.fm/music/punctual,0,"electronic, british, electro"
8936,Luke Faulkner,553389,99914,,https://www.last.fm/music/Luke+Faulkner,0,"Classical, piano, nocturnal"
5848,Automatic,781292,99911,d9377410-255f-40ba-90b8-9cbba6261f4a,https://www.last.fm/music/Automatic,0,"indie rock, indie, british"
5957,The Pogues,18618063,998911,d41a6875-b626-4c0f-89a1-aecb643d29ff,https://www.last.fm/music/The+Pogues,0,"irish, folk, punk"
8047,Erland Cooper,707908,99886,,https://www.last.fm/music/Erland+Cooper,0,"ambient, contemporary classical, modern classical"
7649,Scary Pockets,1977299,99874,,https://www.last.fm/music/Scary+Pockets,0,"funk, seen live, USA"
4210,Eagle-Eye Cherry,8060653,998726,4a5777b3-f55b-437c-8b23-d9ee7791c7fc,https://www.last.fm/music/Eagle-Eye+Cherry,0,"rock, pop, alternative"
2534,SAFETY TRANCE,1802789,99843,,https://www.last.fm/music/SAFETY+TRANCE,0,"neoperreo, electronic, venezuela"
8339,Shawn James,1813055,99778,,https://www.last.fm/music/Shawn+James,0,"soul, folk, blues"


In [33]:
artists.to_csv('artists.csv', index=False)

In [78]:
def lookup_user_playcount(user):
    response = lastfm_get({
        'method': 'user.getinfo',
        'user':  user
    })

    # if there's an error, just return nothing
    if response.status_code != 200:
        return None

    playcount = response.json()['user']['playcount'] 
    
    # rate limiting
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)
    return playcount

In [79]:
lookup_user_playcount('RJ')

'138552'

In [84]:
responses = []

page = 1
total_pages = 2 # this is just a dummy number so the loop starts

while page <= total_pages:
    payload = {
        'method': 'user.getlovedtracks',
        'user': 'RJ',
        'limit': 500,
        'page': page
    }

    # print some output so we can see the status
    print("Requesting page {}/{}".format(page, total_pages))
    # clear the output to make things neater
    clear_output(wait = True)

    # make the API call
    response = lastfm_get(payload)

    # if we get an error, print the response and halt the loop
    if response.status_code != 200:
        print(response.text)
        break

    # extract pagination info
    page = int(response.json()['lovedtracks']['@attr']['page'])
    total_pages = int(response.json()['lovedtracks']['@attr']['totalPages'])

    # append response
    responses.append(response)

    # if it's not a cached result, sleep
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)

    # increment the page number
    page += 1

Requesting page 2/2


In [88]:
r0 = responses[0]
r0_json = r0.json()
r0_lovedtracks = r0_json['lovedtracks']['track']
r0_df = pd.DataFrame(r0_lovedtracks)
r0_df.head()

,artist,date,mbid,url,name,image,streamable
0,{'url': 'https://www.last.fm/music/The+Black+S...,"{'uts': '1459767227', '#text': '04 Apr 2016, 1...",11f098dd-0f6b-32bc-8e68-de7f2facbb62,https://www.last.fm/music/The+Black+Seeds/_/Fire,Fire,"[{'size': 'small', '#text': 'https://lastfm.fr...","{'fulltrack': '0', '#text': '0'}"
1,"{'url': 'https://www.last.fm/music/Gramatik', ...","{'uts': '1457357120', '#text': '07 Mar 2016, 1...",509c51d0-f3ef-4701-b1c5-f7c904917ed5,https://www.last.fm/music/Gramatik/_/Just+Jamm...,Just Jammin',"[{'size': 'small', '#text': 'https://lastfm.fr...","{'fulltrack': '0', '#text': '0'}"
2,{'url': 'https://www.last.fm/music/Tahuna+Brea...,"{'uts': '1443718648', '#text': '01 Oct 2015, 1...",969bbc05-fc94-3c9b-bc13-7eaa3b96774a,https://www.last.fm/music/Tahuna+Breaks/_/Gidd...,Giddy Up,"[{'size': 'small', '#text': 'https://lastfm.fr...","{'fulltrack': '0', '#text': '0'}"
3,{'url': 'https://www.last.fm/music/Fat+Freddy%...,"{'uts': '1441635789', '#text': '07 Sep 2015, 1...",1fe1bb90-fb99-313e-abea-0994467e4e42,https://www.last.fm/music/Fat+Freddy%27s+Drop/...,The Nod,"[{'size': 'small', '#text': 'https://lastfm.fr...","{'fulltrack': '0', '#text': '0'}"
4,{'url': 'https://www.last.fm/music/Gil+Scott-H...,"{'uts': '1441634911', '#text': '07 Sep 2015, 1...",,https://www.last.fm/music/Gil+Scott-Heron/_/La...,Lady Day and John Coltrane,"[{'size': 'small', '#text': 'https://lastfm.fr...","{'fulltrack': '0', '#text': '0'}"


In [34]:
import time

def lookup_user_friends(user):
    response = lastfm_get({
        'method': 'user.getfriends',
        'user':  user
    })

    # if there's an error, just return nothing
    if response.status_code != 200:
        return None

    friends = [t['name'] for t in response.json()['friends']['user']]
    friends_str = ', '.join(friends)
    
    # rate limiting
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)
    return friends_str

In [35]:
friend_list = lookup_user_friends('RJ')

In [36]:
friend_list

'Babs_05, franhale, eartle, massdosage, Knapster01, jonocole, isaac, lobsterclaw, jajo, mremond, Orlenay, schlagschnitzel, Edouard, naniel, dunk, RUPERT, mxcl, jwheare, nancyvw, underpangs, p_wheel, spietsch, musicmobs, Schrollum, luke_10, tgwizard, pkeanecbs, Roelven, BecFrost, gracehn001, saulklein, arrdis, jarvis, fascinated, noirsette, marquezmj, julians, david, pellitero, claoi, grazziee, clairewkyb, lumberjack, foreverautumn, Yllona, cakemix, Jonty, nova77LF, muz, mischa'